In [2]:
#| code-summary: Import module / Set options and theme
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import xml.etree.ElementTree as ET
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import ttest_rel
from statsmodels.stats.weightstats import ttest_ind
import numpy as np
import pingouin as pg
from scipy.stats import zscore
import plotly.graph_objects as go
import pandas as pd
from plotly.subplots import make_subplots
import warnings
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import plotly.express as px
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 10)

In [108]:
#| code-summary: Import cleaned data

df = pd.read_csv('./data/cleaned_df.csv')
df['Location_ID'] = df.groupby(['long', 'lat']).ngroup() + 1

group_list = ['Park', 'long', 'lat', 'veg', 'year', 'TimePeriod', 'RCP','treecanopy', 'Ann_Herb', 'Bare', 'Herb', 'Litter', 'Shrub', 'El', 'Sa','Cl', 'RF', 'Slope', 'E', 'S']
veg_location = df.drop(labels='scenario',axis=1).groupby(group_list).mean().reset_index()
# veg_location['T_Annual'] = (veg_location['T_Annual'] - veg_location['T_Annual'].min()) / (veg_location['T_Annual'].max() - veg_location['T_Annual'].min())


# Average Scenario Dataset
# Convert to numeric, coercing errors to NaN
numeric_series = pd.to_numeric(veg_location['RCP'], errors='coerce')

numeric_series

# Fill NaNs with original non-numeric values
veg_location['RCP'] = numeric_series.fillna(veg_location['RCP'])

four = veg_location[veg_location['RCP'].isin([4.5])]
eight = veg_location[veg_location['RCP'].isin([8.5])]
four_h = veg_location[veg_location['RCP'].isin(['historical'])]
four_h['RCP'] = 4.5
eight_h = veg_location[veg_location['RCP'].isin(['historical'])]
eight_h['RCP'] = 8.5

df_con = pd.concat([four_h, four, eight_h, eight], ignore_index=True)
df_con['Location_ID'] = df_con.groupby(['long', 'lat']).ngroup() + 1


# Scenario Dataset
# Convert to numeric, coercing errors to NaN
numeric_series = pd.to_numeric(df['RCP'], errors='coerce')

numeric_series

# Fill NaNs with original non-numeric values
df['RCP'] = numeric_series.fillna(df['RCP'])

four = df[df['RCP'].isin([4.5])]
eight = df[df['RCP'].isin([8.5])]
four_h = df[df['RCP'].isin(['historical'])]
four_h['RCP'] = 4.5
eight_h = df[df['RCP'].isin(['historical'])]
eight_h['RCP'] = 8.5

df_orig = pd.concat([four_h, four, eight_h, eight], ignore_index=True)
df_orig['Location_ID'] = df_orig.groupby(['long', 'lat']).ngroup() + 1

In [4]:
df_orig

,Park,long,lat,veg,year,TimePeriod,RCP,scenario,treecanopy,Ann_Herb,Bare,Herb,Litter,Shrub,El,Sa,Cl,RF,Slope,E,S,T_P_Corr,DrySoilDays_Winter_top50,DrySoilDays_Spring_top50,DrySoilDays_Summer_top50,DrySoilDays_Fall_top50,DrySoilDays_Winter_whole,DrySoilDays_Spring_whole,DrySoilDays_Summer_whole,DrySoilDays_Fall_whole,Evap_Winter,Evap_Spring,Evap_Summer,Evap_Fall,ExtremeShortTermDryStress_Winter_top50,ExtremeShortTermDryStress_Spring_top50,ExtremeShortTermDryStress_Summer_top50,ExtremeShortTermDryStress_Fall_top50,ExtremeShortTermDryStress_Winter_whole,ExtremeShortTermDryStress_Spring_whole,ExtremeShortTermDryStress_Summer_whole,ExtremeShortTermDryStress_Fall_whole,FrostDays_Winter,FrostDays_Spring,FrostDays_Summer,FrostDays_Fall,NonDrySWA_Winter_top50,NonDrySWA_Spring_top50,NonDrySWA_Summer_top50,NonDrySWA_Fall_top50,NonDrySWA_Winter_whole,NonDrySWA_Spring_whole,NonDrySWA_Summer_whole,NonDrySWA_Fall_whole,PET_Winter,PET_Spring,PET_Summer,PET_Fall,PPT_Winter,PPT_Spring,PPT_Summer,PPT_Fall,SemiDryDuration_Annual_top50,SemiDryDuration_Annual_whole,SWA_Winter_top50,SWA_Spring_top50,SWA_Summer_top50,SWA_Fall_top50,SWA_Winter_whole,SWA_Spring_whole,SWA_Summer_whole,SWA_Fall_whole,T_Winter,T_Spring,T_Summer,T_Fall,Tmax_Winter,Tmax_Spring,Tmax_Summer,Tmax_Fall,Tmin_Winter,Tmin_Spring,Tmin_Summer,Tmin_Fall,Transp_Winter,Transp_Spring,Transp_Summer,Transp_Fall,VWC_Winter_top50,VWC_Spring_top50,VWC_Summer_top50,VWC_Fall_top50,VWC_Winter_whole,VWC_Spring_whole,VWC_Summer_whole,VWC_Fall_whole,WetSoilDays_Winter_top50,WetSoilDays_Spring_top50,WetSoilDays_Summer_top50,WetSoilDays_Fall_top50,WetSoilDays_Winter_whole,WetSoilDays_Spring_whole,WetSoilDays_Summer_whole,WetSoilDays_Fall_whole,PPT_Annual,T_Annual,RL,Location_ID
0,NABR,-110.0472,37.60413,Shrubland,1980,Hist,4.5,sc1,0,0,84,5,11,7,1764.955,77.03307,6.082058,2.285707,1949.283,-8753.784,4834.13,-0.6636760860,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.7140658366,6.3995308949,1.5598074021,3.3632779979,NaN,24.340,36.160,29.520,NaN,24.340,36.160,29.520,75.0,34.0,0.0,26.0,3.4668806371,2.6546632530,0.0321140671,0.4880867481,3.4668806371,2.6546632530,0.0321140671,0.4880867481,7.7811633032,31.1394527955,48.0177480655,21.9156825756,13.79,8.71,2.69,6.37,36.5000000000,36.5000000000,3.4668806371,2.6546632530,0.0321140671,0.4880867481,3.4668806371,2.6546632530,0.0321140671,0.4880867481,0.96483520,8.767935,23.15924,11.962090,14.15,28.75,37.05,31.15,-12.45,-7.35,5.55,-10.25,0.2370806,5.296833,1.0674960,1.9667860,0.1134468701,0.0968307001,0.0418759016,0.0522975530,0.1134468701,0.0968307001,0.0418759016,0.0522975530,91.0,77.0,5.0,47.0,91.0,77.0,5.0,47.0,31.56,11.21352505,54.57202074,1
1,NABR,-110.0472,37.60413,Shrubland,1981,Hist,4.5,sc1,0,0,84,5,11,7,1764.955,77.03307,6.082058,2.285707,1949.283,-8753.784,4834.13,0.3478010620,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1815202084,5.9723378265,5.0428776741,4.6374034668,13.92,26.530,36.080,NaN,13.92,26.530,36.080,NaN,79.0,26.0,0.0,13.0,0.3461917264,0.8982752558,0.0336629893,2.5013360811,0.3461917264,0.8982752558,0.0336629893,2.5013360811,8.1229049607,32.3882557036,48.1772426406,21.7575735702,2.25,9.81,9.39,11.75,13.2500000000,13.2500000000,0.3461917264,0.8982752558,0.0336629893,2.5013360811,0.3461917264,0.8982752558,0.0336629893,2.5013360811,3.33444400,10.548370,23.27065,11.581320,17.05,28.15,37.55,29.75,-9.35,-5.55,1.25,-7.25,0.2930753,3.506108,3.9163280,2.7875470,0.0493818430,0.0607271763,0.0426386771,0.0936706801,0.0493818430,0.0607271763,0.0426386771,0.0936706801,48.0,60.0,13.0,85.0,48.0,60.0,13.0,85.0,33.20,12.18369600,54.57202074,1
2,NABR,-110.0472,37.60413,Shrubland,1982,Hist,4.5,sc1,0,0,84,5,11,7,1764.955,77.03307,6.082058,2.285707,1949.283,-8753.784,4834.13,0.3260300992,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.2589947135,4.7173273934,4.5276363327,4.2477717540,NaN,26.190,34.990,22.060,NaN,26.190,34.990,22.060,83.0,21.0,0.0,30.0,3.2599844936,1.5994982052,0.1993822366,1.2432253150,3.2599844936,1.5994982052,0.1993822366,1.2432253150,7.3379526955,31.4894498184,47.1800768757,21.

In [65]:
temp = df_orig[df_orig['TimePeriod'] != "Hist"].drop(['scenario','year','TimePeriod','veg','Park'],axis=1)

In [66]:
loc = temp.groupby(list(temp.columns[:3])).mean().reset_index()

In [67]:
loc

,long,lat,RCP,treecanopy,Ann_Herb,Bare,Herb,Litter,Shrub,El,Sa,Cl,RF,Slope,E,S,T_P_Corr,DrySoilDays_Winter_top50,DrySoilDays_Spring_top50,DrySoilDays_Summer_top50,DrySoilDays_Fall_top50,DrySoilDays_Winter_whole,DrySoilDays_Spring_whole,DrySoilDays_Summer_whole,DrySoilDays_Fall_whole,Evap_Winter,Evap_Spring,Evap_Summer,Evap_Fall,ExtremeShortTermDryStress_Winter_top50,ExtremeShortTermDryStress_Spring_top50,ExtremeShortTermDryStress_Summer_top50,ExtremeShortTermDryStress_Fall_top50,ExtremeShortTermDryStress_Winter_whole,ExtremeShortTermDryStress_Spring_whole,ExtremeShortTermDryStress_Summer_whole,ExtremeShortTermDryStress_Fall_whole,FrostDays_Winter,FrostDays_Spring,FrostDays_Summer,FrostDays_Fall,NonDrySWA_Winter_top50,NonDrySWA_Spring_top50,NonDrySWA_Summer_top50,NonDrySWA_Fall_top50,NonDrySWA_Winter_whole,NonDrySWA_Spring_whole,NonDrySWA_Summer_whole,NonDrySWA_Fall_whole,PET_Winter,PET_Spring,PET_Summer,PET_Fall,PPT_Winter,PPT_Spring,PPT_Summer,PPT_Fall,SemiDryDuration_Annual_top50,SemiDryDuration_Annual_whole,SWA_Winter_top50,SWA_Spring_top50,SWA_Summer_top50,SWA_Fall_top50,SWA_Winter_whole,SWA_Spring_whole,SWA_Summer_whole,SWA_Fall_whole,T_Winter,T_Spring,T_Summer,T_Fall,Tmax_Winter,Tmax_Spring,Tmax_Summer,Tmax_Fall,Tmin_Winter,Tmin_Spring,Tmin_Summer,Tmin_Fall,Transp_Winter,Transp_Spring,Transp_Summer,Transp_Fall,VWC_Winter_top50,VWC_Spring_top50,VWC_Summer_top50,VWC_Fall_top50,VWC_Winter_whole,VWC_Spring_whole,VWC_Summer_whole,VWC_Fall_whole,WetSoilDays_Winter_top50,WetSoilDays_Spring_top50,WetSoilDays_Summer_top50,WetSoilDays_Fall_top50,WetSoilDays_Winter_whole,WetSoilDays_Spring_whole,WetSoilDays_Summer_whole,WetSoilDays_Fall_whole,PPT_Annual,T_Annual,RL,Location_ID
0,-110.0472,37.60413,4.5,0.0,0.0,84.0,5.0,11.0,7.0,1764.955,77.03307,6.082058,2.285707,1949.283,-8753.784,4834.130,0.0362946938,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0,0.0000000000,0.0000000000,2.9279271385,4.3136374237,3.4006431629,3.8634883769,12.6000274914,30.2888365508,37.9360968354,31.8152392344,12.6000274914,30.2888365508,37.9360968354,31.8152392344,62.6905063291,8.8101265823,0.0000000000,14.1259493671,2.8824872493,1.6620789701,0.1792122427,1.4297005538,2.8824872493,1.6620789701,0.1792122427,1.4297005538,8.0553793094,34.1466680955,49.9296815850,23.0422069326,6.7315189873,6.0797848101,7.5413734177,9.0627721519,28.9093317406,28.9093317406,2.8824872493,1.6620789701,0.1792122427,1.4297005538,2.8824872493,1.6620789701,0.1792122427,1.4297005538,2.5445671242,12.9721604595,25.5563070253,13.9088079146,17.0724746835,32.7155696203,39.1244050633,33.7560506329,-12.1963101266,-3.5686139241,8.1549556962,-6.8100253165,0.3297689865,3.8791521741,3.3871605063,1.9037310404,0.1014413241,0.0760749178,0.0452951895,0.0714751376,0.1014413241,0.0760749178,0.0452951895,0.0714751376,83.3430379747,60.7303797468,13.9867088608,57.2873417722,83.3430379747,60.7303797468,13.9867088608,57.2873417722,29.4154493671,13.7454606309,54.5720207400,1.0
1,-110.0472,37.60413,8.5,0.0,0.0,84.0,5.0,11.0,7.0,1764.955,77.03307,6.082058,2.285707,1949.283,-8753.784,4834.130,0.0212450012,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0,0.0000000000,0.0000000000,2.9082278534,4.1881907153,3.3117269492,3.7867532388,14.5495932836,31.6529495472,39.5574639241,33.2222073171,14.5495932836,31.6529495472,39.5574639241,33.2222073171,51.5132911392,6.7208860759,0.0000000000,11.0025316456,2.9555400375,1.7230310748,0.2094173320,1.4700958165,2.9555400375,1.7230310748,0.2094173320,1.4700958165,8.3877297817,34.9279692424,51.0481062857,23.7633800197,7.0628417722,5.9293037975,7.6618924051,9.0752974684,30.3738682817,30.3738682817,2.9555400375,1.7230310748,0.2094173320,1.4700958165,2.9555400375,1.7230310748,0.2094173320,1.4700958165,3.7744983367,14.0176108696,26.9712245190,15.2732070759,18.4157025316,34.1654683544,40.8539556962,35.3909873418,-9.9839493671,-2.5874113924,9.5043291139,-5.5749810127,0.3460470111,3.8210640110,3.5038976743,1.9859378078,0.1029423837,0.0773132135,0.0459

In [7]:
df_con

,Park,long,lat,veg,year,TimePeriod,RCP,treecanopy,Ann_Herb,Bare,Herb,Litter,Shrub,El,Sa,Cl,RF,Slope,E,S,T_P_Corr,DrySoilDays_Winter_top50,DrySoilDays_Spring_top50,DrySoilDays_Summer_top50,DrySoilDays_Fall_top50,DrySoilDays_Winter_whole,DrySoilDays_Spring_whole,DrySoilDays_Summer_whole,DrySoilDays_Fall_whole,Evap_Winter,Evap_Spring,Evap_Summer,Evap_Fall,ExtremeShortTermDryStress_Winter_top50,ExtremeShortTermDryStress_Spring_top50,ExtremeShortTermDryStress_Summer_top50,ExtremeShortTermDryStress_Fall_top50,ExtremeShortTermDryStress_Winter_whole,ExtremeShortTermDryStress_Spring_whole,ExtremeShortTermDryStress_Summer_whole,ExtremeShortTermDryStress_Fall_whole,FrostDays_Winter,FrostDays_Spring,FrostDays_Summer,FrostDays_Fall,NonDrySWA_Winter_top50,NonDrySWA_Spring_top50,NonDrySWA_Summer_top50,NonDrySWA_Fall_top50,NonDrySWA_Winter_whole,NonDrySWA_Spring_whole,NonDrySWA_Summer_whole,NonDrySWA_Fall_whole,PET_Winter,PET_Spring,PET_Summer,PET_Fall,PPT_Winter,PPT_Spring,PPT_Summer,PPT_Fall,SemiDryDuration_Annual_top50,SemiDryDuration_Annual_whole,SWA_Winter_top50,SWA_Spring_top50,SWA_Summer_top50,SWA_Fall_top50,SWA_Winter_whole,SWA_Spring_whole,SWA_Summer_whole,SWA_Fall_whole,T_Winter,T_Spring,T_Summer,T_Fall,Tmax_Winter,Tmax_Spring,Tmax_Summer,Tmax_Fall,Tmin_Winter,Tmin_Spring,Tmin_Summer,Tmin_Fall,Transp_Winter,Transp_Spring,Transp_Summer,Transp_Fall,VWC_Winter_top50,VWC_Spring_top50,VWC_Summer_top50,VWC_Fall_top50,VWC_Winter_whole,VWC_Spring_whole,VWC_Summer_whole,VWC_Fall_whole,WetSoilDays_Winter_top50,WetSoilDays_Spring_top50,WetSoilDays_Summer_top50,WetSoilDays_Fall_top50,WetSoilDays_Winter_whole,WetSoilDays_Spring_whole,WetSoilDays_Summer_whole,WetSoilDays_Fall_whole,PPT_Annual,T_Annual,RL,Location_ID
0,NABR,-110.0472,37.60413,Shrubland,1980,Hist,4.5,0,0,84,5,11,7,1764.955,77.03307,6.082058,2.285707,1949.283,-8753.784,4834.13,-0.6636760860,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,3.7140658366,6.3995308949,1.5598074021,3.3632779979,NaN,24.3400000000,36.1600000000,29.5200000000,NaN,24.3400000000,36.1600000000,29.5200000000,75.00,34.00,0.0,26.00,3.4668806371,2.6546632530,0.0321140671,0.4880867481,3.4668806371,2.6546632530,0.0321140671,0.4880867481,7.7811633032,31.1394527955,48.0177480655,21.9156825756,13.7900,8.7100,2.6900,6.3700,36.5000000000,36.5000000000,3.4668806371,2.6546632530,0.0321140671,0.4880867481,3.4668806371,2.6546632530,0.0321140671,0.4880867481,0.96483520,8.7679350,23.1592400,11.9620900,14.1500,28.7500,37.0500,31.1500,-12.4500,-7.3500,5.5500,-10.2500,0.2370806000,5.29683300,1.0674960000,1.9667860000,0.1134468701,0.0968307001,0.0418759016,0.0522975530,0.1134468701,0.0968307001,0.0418759016,0.0522975530,91.00,77.00,5.00,47.00,91.00,77.00,5.00,47.00,31.5600,11.2135250500,54.57202074,1
1,NABR,-110.0472,37.60413,Shrubland,1981,Hist,4.5,0,0,84,5,11,7,1764.955,77.03307,6.082058,2.285707,1949.283,-8753.784,4834.13,0.3478010620,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,2.1815202084,5.9723378265,5.0428776741,4.6374034668,13.920,26.5300000000,36.0800000000,NaN,13.920,26.5300000000,36.0800000000,NaN,79.00,26.00,0.0,13.00,0.3461917264,0.8982752558,0.0336629893,2.5013360811,0.3461917264,0.8982752558,0.0336629893,2.5013360811,8.1229049607,32.3882557036,48.1772426406,21.7575735702,2.2500,9.8100,9.3900,11.7500,13.2500000000,13.2500000000,0.3461917264,0.8982752558,0.0336629893,2.5013360811,0.3461917264,0.8982752558,0.0336629893,2.5013360811,3.33444400,10.5483700,23.2706500,11.5813200,17.0500,28.1500,37.5500,29.7500,-9.3500,-5.5500,1.2500,-7.2500,0.2930753000,3.50610800,3.9163280000,2.7875470000,0.0493818430,0.0607271763,0.0426386771,0.0936706801,0.0493818430,0.0607271763,0.0426386771,0.0936706801,48.00,60.00,13.00,85.00,48.00,60.00,13.00,85.00,33.2000,12.1836960000,54.57202074,1
2,NABR,-110.0472,37.60413,Shrubland,1982,Hist,4.5,0,0,84,5,11,7,1764.955,77.03307,6.082058,2.285707,1949.283,-8753.784,4834.13,0.3260300992,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,3.2589947135,4.7173273934,4.5276363327,4.2477717540,NaN,26.1900000000,34.9900000000,22.06000

In [8]:
loc

,Park,long,lat,veg,year,TimePeriod,RCP,treecanopy,Ann_Herb,Bare,Herb,Litter,Shrub,El,Sa,Cl,RF,Slope,E,S,T_P_Corr,DrySoilDays_Winter_top50,DrySoilDays_Spring_top50,DrySoilDays_Summer_top50,DrySoilDays_Fall_top50,DrySoilDays_Winter_whole,DrySoilDays_Spring_whole,DrySoilDays_Summer_whole,DrySoilDays_Fall_whole,Evap_Winter,Evap_Spring,Evap_Summer,Evap_Fall,ExtremeShortTermDryStress_Winter_top50,ExtremeShortTermDryStress_Spring_top50,ExtremeShortTermDryStress_Summer_top50,ExtremeShortTermDryStress_Fall_top50,ExtremeShortTermDryStress_Winter_whole,ExtremeShortTermDryStress_Spring_whole,ExtremeShortTermDryStress_Summer_whole,ExtremeShortTermDryStress_Fall_whole,FrostDays_Winter,FrostDays_Spring,FrostDays_Summer,FrostDays_Fall,NonDrySWA_Winter_top50,NonDrySWA_Spring_top50,NonDrySWA_Summer_top50,NonDrySWA_Fall_top50,NonDrySWA_Winter_whole,NonDrySWA_Spring_whole,NonDrySWA_Summer_whole,NonDrySWA_Fall_whole,PET_Winter,PET_Spring,PET_Summer,PET_Fall,PPT_Winter,PPT_Spring,PPT_Summer,PPT_Fall,SemiDryDuration_Annual_top50,SemiDryDuration_Annual_whole,SWA_Winter_top50,SWA_Spring_top50,SWA_Summer_top50,SWA_Fall_top50,SWA_Winter_whole,SWA_Spring_whole,SWA_Summer_whole,SWA_Fall_whole,T_Winter,T_Spring,T_Summer,T_Fall,Tmax_Winter,Tmax_Spring,Tmax_Summer,Tmax_Fall,Tmin_Winter,Tmin_Spring,Tmin_Summer,Tmin_Fall,Transp_Winter,Transp_Spring,Transp_Summer,Transp_Fall,VWC_Winter_top50,VWC_Spring_top50,VWC_Summer_top50,VWC_Fall_top50,VWC_Winter_whole,VWC_Spring_whole,VWC_Summer_whole,VWC_Fall_whole,WetSoilDays_Winter_top50,WetSoilDays_Spring_top50,WetSoilDays_Summer_top50,WetSoilDays_Fall_top50,WetSoilDays_Winter_whole,WetSoilDays_Spring_whole,WetSoilDays_Summer_whole,WetSoilDays_Fall_whole,PPT_Annual,T_Annual,RL,Location_ID
0,NABR,-110.0472,37.60413,Shrubland,1980,Hist,4.5,0,0,84,5,11,7,1764.955,77.03307,6.082058,2.285707,1949.283,-8753.784,4834.13,-0.6636760860,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,3.7140658366,6.3995308949,1.5598074021,3.3632779979,NaN,24.3400000000,36.1600000000,29.5200000000,NaN,24.34000,36.1600000000,29.5200000000,75.00,34.00,0.0,26.00,3.4668806371,2.6546632530,0.0321140671,0.4880867481,3.4668806371,2.6546632530,0.0321140671,0.4880867481,7.7811633032,31.1394527955,48.0177480655,21.9156825756,13.7900,8.7100,2.6900,6.3700,36.5000000000,36.5000000000,3.4668806371,2.6546632530,0.0321140671,0.4880867481,3.4668806371,2.6546632530,0.0321140671,0.4880867481,0.964835200,8.7679350,23.1592400,11.9620900,14.1500,28.7500,37.0500,31.1500,-12.4500,-7.3500,5.5500,-10.2500,0.237080600,5.29683300,1.067496000,1.9667860000,0.1134468701,0.0968307001,0.0418759016,0.0522975530,0.1134468701,0.0968307001,0.0418759016,0.0522975530,91.00,77.00,5.00,47.00,91.00,77.00,5.00,47.0,31.5600,11.2135250500,54.57202074,1.0
1,NABR,-110.0472,37.60413,Shrubland,1980,Hist,8.5,0,0,84,5,11,7,1764.955,77.03307,6.082058,2.285707,1949.283,-8753.784,4834.13,-0.6636760860,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,3.7140658366,6.3995308949,1.5598074021,3.3632779979,NaN,24.3400000000,36.1600000000,29.5200000000,NaN,24.34000,36.1600000000,29.5200000000,75.00,34.00,0.0,26.00,3.4668806371,2.6546632530,0.0321140671,0.4880867481,3.4668806371,2.6546632530,0.0321140671,0.4880867481,7.7811633032,31.1394527955,48.0177480655,21.9156825756,13.7900,8.7100,2.6900,6.3700,36.5000000000,36.5000000000,3.4668806371,2.6546632530,0.0321140671,0.4880867481,3.4668806371,2.6546632530,0.0321140671,0.4880867481,0.964835200,8.7679350,23.1592400,11.9620900,14.1500,28.7500,37.0500,31.1500,-12.4500,-7.3500,5.5500,-10.2500,0.237080600,5.29683300,1.067496000,1.9667860000,0.1134468701,0.0968307001,0.0418759016,0.0522975530,0.1134468701,0.0968307001,0.0418759016,0.0522975530,91.00,77.00,5.00,47.00,91.00,77.00,5.00,47.0,31.5600,11.2135250500,54.57202074,1.0
2,NABR,-110.0472,37.60413,Shrubland,1981,Hist,4.5,0,0,84,5,11,7,1764.955,77.03307,6.082058,2.285707,1949.283,-8753.784,4834.13,0.3478010620,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,2.1815202084,5.9723378265,5.0428776741,4.6374034668,13.920,26.5300000000,36.0800000000,NaN,13.

In [18]:
#| code-summary: PCA(RCP = 4.5)


In [64]:
loc

,Park,long,lat,RCP,treecanopy,Ann_Herb,Bare,Herb,Litter,Shrub,El,Sa,Cl,RF,Slope,E,S,T_P_Corr,DrySoilDays_Winter_top50,DrySoilDays_Spring_top50,DrySoilDays_Summer_top50,DrySoilDays_Fall_top50,DrySoilDays_Winter_whole,DrySoilDays_Spring_whole,DrySoilDays_Summer_whole,DrySoilDays_Fall_whole,Evap_Winter,Evap_Spring,Evap_Summer,Evap_Fall,ExtremeShortTermDryStress_Winter_top50,ExtremeShortTermDryStress_Spring_top50,ExtremeShortTermDryStress_Summer_top50,ExtremeShortTermDryStress_Fall_top50,ExtremeShortTermDryStress_Winter_whole,ExtremeShortTermDryStress_Spring_whole,ExtremeShortTermDryStress_Summer_whole,ExtremeShortTermDryStress_Fall_whole,FrostDays_Winter,FrostDays_Spring,FrostDays_Summer,FrostDays_Fall,NonDrySWA_Winter_top50,NonDrySWA_Spring_top50,NonDrySWA_Summer_top50,NonDrySWA_Fall_top50,NonDrySWA_Winter_whole,NonDrySWA_Spring_whole,NonDrySWA_Summer_whole,NonDrySWA_Fall_whole,PET_Winter,PET_Spring,PET_Summer,PET_Fall,PPT_Winter,PPT_Spring,PPT_Summer,PPT_Fall,SemiDryDuration_Annual_top50,SemiDryDuration_Annual_whole,SWA_Winter_top50,SWA_Spring_top50,SWA_Summer_top50,SWA_Fall_top50,SWA_Winter_whole,SWA_Spring_whole,SWA_Summer_whole,SWA_Fall_whole,T_Winter,T_Spring,T_Summer,T_Fall,Tmax_Winter,Tmax_Spring,Tmax_Summer,Tmax_Fall,Tmin_Winter,Tmin_Spring,Tmin_Summer,Tmin_Fall,Transp_Winter,Transp_Spring,Transp_Summer,Transp_Fall,VWC_Winter_top50,VWC_Spring_top50,VWC_Summer_top50,VWC_Fall_top50,VWC_Winter_whole,VWC_Spring_whole,VWC_Summer_whole,VWC_Fall_whole,WetSoilDays_Winter_top50,WetSoilDays_Spring_top50,WetSoilDays_Summer_top50,WetSoilDays_Fall_top50,WetSoilDays_Winter_whole,WetSoilDays_Spring_whole,WetSoilDays_Summer_whole,WetSoilDays_Fall_whole,PPT_Annual,T_Annual,RL,Location_ID
0,NABR,-110.0472,37.60413,6.5,0.0,0.0,84.0,5.0,11.0,7.0,1764.955,77.03307,6.082058,2.285707,1949.2830,-8753.78400,4834.130,0.0287698475,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,2.9180774959,4.2509140695,3.3561850561,3.8251208078,13.5347030411,30.9691329032,38.7467803797,32.5119263285,13.5347030411,30.9691329032,38.7467803797,32.5119263285,57.1018987342,7.7655063291,0.0000000000,12.5642405063,2.9190136434,1.6925550225,0.1943147873,1.4498981851,2.9190136434,1.6925550225,0.1943147873,1.4498981851,8.2215545456,34.5373186689,50.4888939354,23.4027934762,6.8971803797,6.0045443038,7.6016329114,9.0690348101,29.6416000111,29.6416000111,2.9190136434,1.6925550225,0.1943147873,1.4498981851,2.9190136434,1.6925550225,0.1943147873,1.4498981851,3.1595327305,13.4948856646,26.2637657722,14.5910074953,17.7440886076,33.4405189873,39.9891803797,34.5735189873,-11.0901297468,-3.0780126582,8.8296424051,-6.1925031646,0.3379079988,3.8501080925,3.4455290903,1.9448344241,0.1021918539,0.0766940657,0.0456196797,0.0718871218,0.1021918539,0.0766940657,0.0456196797,0.0718871218,83.7240506329,60.9332278481,14.5174050633,57.6268987342,83.7240506329,60.9332278481,14.5174050633,57.6268987342,29.5723924051,14.3772979156,54.5720207400,1.0
1,NABR,-110.0461,37.60334,6.5,6.0,0.0,77.0,7.0,6.0,10.0,1772.476,76.05293,7.145646,2.367719,1788.6000,-9006.74600,4343.799,0.0287698477,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,2.8479713415,4.0429059019,3.4103011183,3.6643981296,13.8550756410,31.0549977757,38.7464759494,32.1897231680,13.8550756410,31.0549977757,38.7464759494,32.1897231680,57.1018987342,7.7655063291,0.0000000000,12.5642405063,2.2512185714,0.9958599526,0.1786154147,1.1289614504,2.2512185714,0.9958599526,0.1786154147,1.1289614504,7.9612976981,34.8597629807,51.2447311922,23.2574675622,6.8971803797,6.0045443038,7.6016329114,9.0690348101,29.1716443906,29.1716443906,2.2512185714,0.9958599526,0.1786154147,1.1289614504,2.2512185714,0.9958599526,0.1786154147,1.1289614504,3.1595327305,13.4948856646,26.2637657722,14.5910074953,17.7440886076,33.4405189873,39.9891803797,34.5735189873,-11.0901297468,-3.0780126582,8.8296424051,-6.1925031646,0.9064953867,3.5688694387,3.4191325208,2.541

In [68]:
loc

,long,lat,RCP,treecanopy,Ann_Herb,Bare,Herb,Litter,Shrub,El,Sa,Cl,RF,Slope,E,S,T_P_Corr,DrySoilDays_Winter_top50,DrySoilDays_Spring_top50,DrySoilDays_Summer_top50,DrySoilDays_Fall_top50,DrySoilDays_Winter_whole,DrySoilDays_Spring_whole,DrySoilDays_Summer_whole,DrySoilDays_Fall_whole,Evap_Winter,Evap_Spring,Evap_Summer,Evap_Fall,ExtremeShortTermDryStress_Winter_top50,ExtremeShortTermDryStress_Spring_top50,ExtremeShortTermDryStress_Summer_top50,ExtremeShortTermDryStress_Fall_top50,ExtremeShortTermDryStress_Winter_whole,ExtremeShortTermDryStress_Spring_whole,ExtremeShortTermDryStress_Summer_whole,ExtremeShortTermDryStress_Fall_whole,FrostDays_Winter,FrostDays_Spring,FrostDays_Summer,FrostDays_Fall,NonDrySWA_Winter_top50,NonDrySWA_Spring_top50,NonDrySWA_Summer_top50,NonDrySWA_Fall_top50,NonDrySWA_Winter_whole,NonDrySWA_Spring_whole,NonDrySWA_Summer_whole,NonDrySWA_Fall_whole,PET_Winter,PET_Spring,PET_Summer,PET_Fall,PPT_Winter,PPT_Spring,PPT_Summer,PPT_Fall,SemiDryDuration_Annual_top50,SemiDryDuration_Annual_whole,SWA_Winter_top50,SWA_Spring_top50,SWA_Summer_top50,SWA_Fall_top50,SWA_Winter_whole,SWA_Spring_whole,SWA_Summer_whole,SWA_Fall_whole,T_Winter,T_Spring,T_Summer,T_Fall,Tmax_Winter,Tmax_Spring,Tmax_Summer,Tmax_Fall,Tmin_Winter,Tmin_Spring,Tmin_Summer,Tmin_Fall,Transp_Winter,Transp_Spring,Transp_Summer,Transp_Fall,VWC_Winter_top50,VWC_Spring_top50,VWC_Summer_top50,VWC_Fall_top50,VWC_Winter_whole,VWC_Spring_whole,VWC_Summer_whole,VWC_Fall_whole,WetSoilDays_Winter_top50,WetSoilDays_Spring_top50,WetSoilDays_Summer_top50,WetSoilDays_Fall_top50,WetSoilDays_Winter_whole,WetSoilDays_Spring_whole,WetSoilDays_Summer_whole,WetSoilDays_Fall_whole,PPT_Annual,T_Annual,RL,Location_ID
0,-110.0472,37.60413,4.5,0.0,0.0,84.0,5.0,11.0,7.0,1764.955,77.03307,6.082058,2.285707,1949.283,-8753.784,4834.130,0.0362946938,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0,0.0000000000,0.0000000000,2.9279271385,4.3136374237,3.4006431629,3.8634883769,12.6000274914,30.2888365508,37.9360968354,31.8152392344,12.6000274914,30.2888365508,37.9360968354,31.8152392344,62.6905063291,8.8101265823,0.0000000000,14.1259493671,2.8824872493,1.6620789701,0.1792122427,1.4297005538,2.8824872493,1.6620789701,0.1792122427,1.4297005538,8.0553793094,34.1466680955,49.9296815850,23.0422069326,6.7315189873,6.0797848101,7.5413734177,9.0627721519,28.9093317406,28.9093317406,2.8824872493,1.6620789701,0.1792122427,1.4297005538,2.8824872493,1.6620789701,0.1792122427,1.4297005538,2.5445671242,12.9721604595,25.5563070253,13.9088079146,17.0724746835,32.7155696203,39.1244050633,33.7560506329,-12.1963101266,-3.5686139241,8.1549556962,-6.8100253165,0.3297689865,3.8791521741,3.3871605063,1.9037310404,0.1014413241,0.0760749178,0.0452951895,0.0714751376,0.1014413241,0.0760749178,0.0452951895,0.0714751376,83.3430379747,60.7303797468,13.9867088608,57.2873417722,83.3430379747,60.7303797468,13.9867088608,57.2873417722,29.4154493671,13.7454606309,54.5720207400,1.0
1,-110.0472,37.60413,8.5,0.0,0.0,84.0,5.0,11.0,7.0,1764.955,77.03307,6.082058,2.285707,1949.283,-8753.784,4834.130,0.0212450012,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0,0.0000000000,0.0000000000,2.9082278534,4.1881907153,3.3117269492,3.7867532388,14.5495932836,31.6529495472,39.5574639241,33.2222073171,14.5495932836,31.6529495472,39.5574639241,33.2222073171,51.5132911392,6.7208860759,0.0000000000,11.0025316456,2.9555400375,1.7230310748,0.2094173320,1.4700958165,2.9555400375,1.7230310748,0.2094173320,1.4700958165,8.3877297817,34.9279692424,51.0481062857,23.7633800197,7.0628417722,5.9293037975,7.6618924051,9.0752974684,30.3738682817,30.3738682817,2.9555400375,1.7230310748,0.2094173320,1.4700958165,2.9555400375,1.7230310748,0.2094173320,1.4700958165,3.7744983367,14.0176108696,26.9712245190,15.2732070759,18.4157025316,34.1654683544,40.8539556962,35.3909873418,-9.9839493671,-2.5874113924,9.5043291139,-5.5749810127,0.3460470111,3.8210640110,3.5038976743,1.9859378078,0.1029423837,0.0773132135,0.0459

In [69]:
#| code-summary: Varience Ratio
data = loc[(loc['RCP']==4.5)].dropna(axis=1, how='any')
X = data.iloc[:,list(range(3, len(data.columns)))]

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply PCA
pca = PCA(n_components=10)  # Reduce to 2 components for visualization
X_pca = pca.fit_transform(X_scaled)

# Add the PCA and cluster results to the DataFrame
data['PCA1'] = X_pca[:, 0]
data['PCA2'] = X_pca[:, 1]
data['PCA3'] = X_pca[:, 2]

# Get the component loadings
loadings = pca.components_.T
columns = X.columns

# Create a DataFrame for loadings
loadings_df = pd.DataFrame(loadings, columns=['PCA1', 'PCA2', 'PCA3', 'PCA4', 'PCA5', 'PCA6', 'PCA7', 'PCA8', 'PCA9', 'PCA10'], index=columns)

# Get the explained variance ratio
explained_variance_ratio = pca.explained_variance_ratio_

# Cumulative explained variance
cumulative_explained_variance = np.cumsum(explained_variance_ratio)


# Create a list of x-axis labels from PCA1 to PCA10
x_labels = [f'PCA{i+1}' for i in range(len(explained_variance_ratio))]

# Create a line chart for explained variance ratio
fig = go.Figure(data=go.Scatter(
    x=x_labels,
    y=explained_variance_ratio,
    mode='lines+markers',
    text=explained_variance_ratio,
    textposition='top center'
))

# Update layout for better readability
fig.update_layout(
    title='<b>Explained Variance Ratio by Principal Components</b>',
    xaxis_title='Principal Components',
    yaxis_title='Explained Variance Ratio',
    yaxis=dict(tickformat=".2%", range=[0, 1.1 * explained_variance_ratio.max()]),  # Adjust the range as needed
    template='plotly_white',
    margin=dict(l=50, r=50, b=50, t=50)  # Adjust the padding as needed
)

fig.show()

In [70]:
#| code-summary: Feature Importance Plots

# Sort the features by the absolute value of the loading for PCA1
sorted_loadings = loadings_df['PCA1'].abs().sort_values(ascending=False)

# Get the top 10 most influential features
top_features = sorted_loadings.head(30).index

# Get the actual loadings for these top 10 features
top_loadings = round(loadings_df.loc[top_features, 'PCA1'],4)

# Create a color list based on the sign of the loadings
colors = ['blue' if val > 0 else 'red' for val in top_loadings]

# Create a bar chart
fig = go.Figure(data=[go.Bar(
    x=top_loadings.index,
    y=top_loadings.abs(),
    text=top_loadings.values,  # Show the actual values as text
    textposition='inside',
    marker_color=colors,
    showlegend=False
)])

# Add legend manually
fig.add_trace(go.Bar(
    x=[None], y=[None],
    marker=dict(color='blue'),
    showlegend=True,
    name='Positive'
))
fig.add_trace(go.Bar(
    x=[None], y=[None],
    marker=dict(color='red'),
    showlegend=True,
    name='Negative'
))

# Update layout for better readability
fig.update_layout(
    title='<b>Top 20 Most Influential Features on PCA1</b>',
    xaxis_title='Features',
    yaxis_title='Absolute PCA1 Loadings',
    yaxis=dict(tickformat=".2f"),
    template='plotly_white',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    xaxis=dict(tickangle=45),
    margin=dict(l=20, r=20, b=20, t=60)
)

fig.show()

# Sort the features by the absolute value of the loading for PCA1
sorted_loadings = loadings_df['PCA2'].abs().sort_values(ascending=False)

# Get the top 10 most influential features
top_features = sorted_loadings.head(30).index

# Get the actual loadings for these top 10 features
top_loadings = round(loadings_df.loc[top_features, 'PCA2'],4)

# Create a color list based on the sign of the loadings
colors = ['blue' if val > 0 else 'red' for val in top_loadings]

# Create a bar chart
fig = go.Figure(data=[go.Bar(
    x=top_loadings.index,
    y=top_loadings.abs(),
    text=top_loadings.values,  # Show the actual values as text
    textposition='inside',
    marker_color=colors,
    showlegend=False
)])

# Add legend manually
fig.add_trace(go.Bar(
    x=[None], y=[None],
    marker=dict(color='blue'),
    showlegend=True,
    name='Positive'
))
fig.add_trace(go.Bar(
    x=[None], y=[None],
    marker=dict(color='red'),
    showlegend=True,
    name='Negative'
))

# Update layout for better readability
fig.update_layout(
    title='<b>Top 20 Most Influential Features on PCA2</b>',
    xaxis_title='Features',
    yaxis_title='Absolute PCA1 Loadings',
    yaxis=dict(tickformat=".2f"),
    template='plotly_white',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    xaxis=dict(tickangle=45),
    margin=dict(l=20, r=20, b=20, t=60)
)

fig.show()

# Sort the features by the absolute value of the loading for PCA1
sorted_loadings = loadings_df['PCA3'].abs().sort_values(ascending=False)

# Get the top 10 most influential features
top_features = sorted_loadings.head(30).index

# Get the actual loadings for these top 10 features
top_loadings = round(loadings_df.loc[top_features, 'PCA3'],4)

# Create a color list based on the sign of the loadings
colors = ['blue' if val > 0 else 'red' for val in top_loadings]

# Create a bar chart
fig = go.Figure(data=[go.Bar(
    x=top_loadings.index,
    y=top_loadings.abs(),
    text=top_loadings.values,  # Show the actual values as text
    textposition='inside',
    marker_color=colors,
    showlegend=False
)])

# Add legend manually
fig.add_trace(go.Bar(
    x=[None], y=[None],
    marker=dict(color='blue'),
    showlegend=True,
    name='Positive'
))
fig.add_trace(go.Bar(
    x=[None], y=[None],
    marker=dict(color='red'),
    showlegend=True,
    name='Negative'
))

# Update layout for better readability
fig.update_layout(
    title='<b>Top 20 Most Influential Features on PCA3</b>',
    xaxis_title='Features',
    yaxis_title='Absolute PCA1 Loadings',
    yaxis=dict(tickformat=".2f"),
    template='plotly_white',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    xaxis=dict(tickangle=45),
    margin=dict(l=20, r=20, b=20, t=60)
)

fig.show()

In [86]:
data

,long,lat,RCP,treecanopy,Ann_Herb,Bare,Herb,Litter,Shrub,El,Sa,Cl,RF,Slope,E,S,T_P_Corr,DrySoilDays_Winter_top50,DrySoilDays_Spring_top50,DrySoilDays_Summer_top50,DrySoilDays_Fall_top50,DrySoilDays_Winter_whole,DrySoilDays_Spring_whole,DrySoilDays_Summer_whole,DrySoilDays_Fall_whole,Evap_Winter,Evap_Spring,Evap_Summer,Evap_Fall,ExtremeShortTermDryStress_Winter_top50,ExtremeShortTermDryStress_Spring_top50,ExtremeShortTermDryStress_Summer_top50,ExtremeShortTermDryStress_Fall_top50,ExtremeShortTermDryStress_Winter_whole,ExtremeShortTermDryStress_Spring_whole,ExtremeShortTermDryStress_Summer_whole,ExtremeShortTermDryStress_Fall_whole,FrostDays_Winter,FrostDays_Spring,FrostDays_Summer,FrostDays_Fall,NonDrySWA_Winter_top50,NonDrySWA_Spring_top50,NonDrySWA_Summer_top50,NonDrySWA_Fall_top50,NonDrySWA_Winter_whole,NonDrySWA_Spring_whole,NonDrySWA_Summer_whole,NonDrySWA_Fall_whole,PET_Winter,PET_Spring,PET_Summer,PET_Fall,PPT_Winter,PPT_Spring,PPT_Summer,PPT_Fall,SemiDryDuration_Annual_top50,SemiDryDuration_Annual_whole,SWA_Winter_top50,SWA_Spring_top50,SWA_Summer_top50,SWA_Fall_top50,SWA_Winter_whole,SWA_Spring_whole,SWA_Summer_whole,SWA_Fall_whole,T_Winter,T_Spring,T_Summer,T_Fall,Tmax_Winter,Tmax_Spring,Tmax_Summer,Tmax_Fall,Tmin_Winter,Tmin_Spring,Tmin_Summer,Tmin_Fall,Transp_Winter,Transp_Spring,Transp_Summer,Transp_Fall,VWC_Winter_top50,VWC_Spring_top50,VWC_Summer_top50,VWC_Fall_top50,VWC_Winter_whole,VWC_Spring_whole,VWC_Summer_whole,VWC_Fall_whole,WetSoilDays_Winter_top50,WetSoilDays_Spring_top50,WetSoilDays_Summer_top50,WetSoilDays_Fall_top50,WetSoilDays_Winter_whole,WetSoilDays_Spring_whole,WetSoilDays_Summer_whole,WetSoilDays_Fall_whole,PPT_Annual,T_Annual,RL,Location_ID,PCA1,PCA2,PCA3
0,-110.0472,37.60413,4.5,0.0,0.0,84.0,5.0,11.0,7.0,1764.955,77.03307,6.082058,2.285707,1949.2830,-8753.78400,4834.130,0.0362946938,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,2.9279271385,4.3136374237,3.4006431629,3.8634883769,12.6000274914,30.2888365508,37.9360968354,31.8152392344,12.6000274914,30.2888365508,37.9360968354,31.8152392344,62.6905063291,8.8101265823,0.0000000000,14.1259493671,2.8824872493,1.6620789701,0.1792122427,1.4297005538,2.8824872493,1.6620789701,0.1792122427,1.4297005538,8.0553793094,34.1466680955,49.9296815850,23.0422069326,6.7315189873,6.0797848101,7.5413734177,9.0627721519,28.9093317406,28.9093317406,2.8824872493,1.6620789701,0.1792122427,1.4297005538,2.8824872493,1.6620789701,0.1792122427,1.4297005538,2.5445671242,12.9721604595,25.5563070253,13.9088079146,17.0724746835,32.7155696203,39.1244050633,33.7560506329,-12.1963101266,-3.5686139241,8.1549556962,-6.8100253165,0.3297689865,3.8791521741,3.3871605063,1.9037310404,0.1014413241,0.0760749178,0.0452951895,0.0714751376,0.1014413241,0.0760749178,0.0452951895,0.0714751376,83.3430379747,60.7303797468,13.9867088608,57.2873417722,83.3430379747,60.7303797468,13.9867088608,57.2873417722,29.4154493671,13.7454606309,54.5720207400,1.0,-7.4239654667,1.3766184705,-1.7939681014
2,-110.0461,37.60334,4.5,6.0,0.0,77.0,7.0,6.0,10.0,1772.476,76.05293,7.145646,2.367719,1788.6000,-9006.74600,4343.799,0.0362946942,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,2.8587175167,4.0908267843,3.4409474631,3.6920371235,13.1231367089,30.3801498413,37.9356056962,31.4862960526,13.1231367089,30.3801498413,37.9356056962,31.4862960526,62.6905063291,8.8101265823,0.0000000000,14.1259493671,2.2137413596,0.9681282111,0.1663126168,1.1125387199,2.2137413596,0.9681282111,0.1663126168,1.1125387199,7.7986306955,34.4666029095,50.6814379069,22.8984912606,6.7315189873,6.0797848101,7.5413734177,9.0627721519,28.4737752194,28.4737752194,2.2137413596,0.9681282111,0.1663126168,1.1125387199,2.2137413596,0.9681282111,0.1663126168,1.1125387199,2.5445671242,12.9721604595,25.5563070253,13.9088079146,17.0724746835,32.7155696203,39.1244050633,33.7560506329,-12.1963101266,-3.5686139241,8.1549556962,-6.8100253165,0.9

In [85]:
#| code-summary: 2D PCA Plots

# Visualize the results with Plotly
fig = px.scatter(
    data,
    x='PCA1',
    y='PCA2',
    color='veg',
    title='<b>PCA For All Scenarios (RCP = 4.5)</b>',
    labels={'PCA1': 'PCA1', 'PCA2': 'PCA2'},
    opacity=0.5
)

fig.update_layout(
    margin=dict(l=10, r=10, b=10, t=40),  # Adjust the values as needed
    title_x=0.5,
    title_y=0.95,
)

fig.show()

fig = px.scatter_3d(
    data,
    x='PCA1',
    y='PCA2',
    z='PCA3',
    color='veg',
    title='<b>3D PCA For All Scenarios</b>',
    labels={'PCA1': 'PCA1', 'PCA2': 'PCA2', 'PCA3': 'PCA3'},
    opacity=0.5,
    size_max=0.1
)

fig.update_traces(marker=dict(size=3))  # Adjust the size value as needed


# Update layout to adjust padding and margins
fig.update_layout(
    margin=dict(l=5, r=5, b=5, t=20),  # Adjust the values as needed
    title_x=0.5,
    title_y=0.95,
    scene=dict(
        xaxis=dict(title='PCA1'),
        yaxis=dict(title='PCA2'),
        zaxis=dict(title='PCA3'),
                camera=dict(
            eye=dict(x=0, y=2, z=0)
                )
    )
)
fig.show()

ValueError: Value of 'color' is not the name of a column in 'data_frame'. Expected one of ['long', 'lat', 'RCP', 'treecanopy', 'Ann_Herb', 'Bare', 'Herb', 'Litter', 'Shrub', 'El', 'Sa', 'Cl', 'RF', 'Slope', 'E', 'S', 'T_P_Corr', 'DrySoilDays_Winter_top50', 'DrySoilDays_Spring_top50', 'DrySoilDays_Summer_top50', 'DrySoilDays_Fall_top50', 'DrySoilDays_Winter_whole', 'DrySoilDays_Spring_whole', 'DrySoilDays_Summer_whole', 'DrySoilDays_Fall_whole', 'Evap_Winter', 'Evap_Spring', 'Evap_Summer', 'Evap_Fall', 'ExtremeShortTermDryStress_Winter_top50', 'ExtremeShortTermDryStress_Spring_top50', 'ExtremeShortTermDryStress_Summer_top50', 'ExtremeShortTermDryStress_Fall_top50', 'ExtremeShortTermDryStress_Winter_whole', 'ExtremeShortTermDryStress_Spring_whole', 'ExtremeShortTermDryStress_Summer_whole', 'ExtremeShortTermDryStress_Fall_whole', 'FrostDays_Winter', 'FrostDays_Spring', 'FrostDays_Summer', 'FrostDays_Fall', 'NonDrySWA_Winter_top50', 'NonDrySWA_Spring_top50', 'NonDrySWA_Summer_top50', 'NonD

In [27]:
loc.iloc[:,20:]

,T_P_Corr,DrySoilDays_Winter_top50,DrySoilDays_Spring_top50,DrySoilDays_Summer_top50,DrySoilDays_Fall_top50,DrySoilDays_Winter_whole,DrySoilDays_Spring_whole,DrySoilDays_Summer_whole,DrySoilDays_Fall_whole,Evap_Winter,Evap_Spring,Evap_Summer,Evap_Fall,ExtremeShortTermDryStress_Winter_top50,ExtremeShortTermDryStress_Spring_top50,ExtremeShortTermDryStress_Summer_top50,ExtremeShortTermDryStress_Fall_top50,ExtremeShortTermDryStress_Winter_whole,ExtremeShortTermDryStress_Spring_whole,ExtremeShortTermDryStress_Summer_whole,ExtremeShortTermDryStress_Fall_whole,FrostDays_Winter,FrostDays_Spring,FrostDays_Summer,FrostDays_Fall,NonDrySWA_Winter_top50,NonDrySWA_Spring_top50,NonDrySWA_Summer_top50,NonDrySWA_Fall_top50,NonDrySWA_Winter_whole,NonDrySWA_Spring_whole,NonDrySWA_Summer_whole,NonDrySWA_Fall_whole,PET_Winter,PET_Spring,PET_Summer,PET_Fall,PPT_Winter,PPT_Spring,PPT_Summer,PPT_Fall,SemiDryDuration_Annual_top50,SemiDryDuration_Annual_whole,SWA_Winter_top50,SWA_Spring_top50,SWA_Summer_top50,SWA_Fall_top50,SWA_Winter_whole,SWA_Spring_whole,SWA_Summer_whole,SWA_Fall_whole,T_Winter,T_Spring,T_Summer,T_Fall,Tmax_Winter,Tmax_Spring,Tmax_Summer,Tmax_Fall,Tmin_Winter,Tmin_Spring,Tmin_Summer,Tmin_Fall,Transp_Winter,Transp_Spring,Transp_Summer,Transp_Fall,VWC_Winter_top50,VWC_Spring_top50,VWC_Summer_top50,VWC_Fall_top50,VWC_Winter_whole,VWC_Spring_whole,VWC_Summer_whole,VWC_Fall_whole,WetSoilDays_Winter_top50,WetSoilDays_Spring_top50,WetSoilDays_Summer_top50,WetSoilDays_Fall_top50,WetSoilDays_Winter_whole,WetSoilDays_Spring_whole,WetSoilDays_Summer_whole,WetSoilDays_Fall_whole,PPT_Annual,T_Annual,RL,Location_ID
0,-0.6636760860,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,3.7140658366,6.3995308949,1.5598074021,3.3632779979,NaN,24.3400000000,36.1600000000,29.5200000000,NaN,24.34000,36.1600000000,29.5200000000,75.00,34.00,0.0,26.00,3.4668806371,2.6546632530,0.0321140671,0.4880867481,3.4668806371,2.6546632530,0.0321140671,0.4880867481,7.7811633032,31.1394527955,48.0177480655,21.9156825756,13.7900,8.7100,2.6900,6.3700,36.5000000000,36.5000000000,3.4668806371,2.6546632530,0.0321140671,0.4880867481,3.4668806371,2.6546632530,0.0321140671,0.4880867481,0.964835200,8.7679350,23.1592400,11.9620900,14.1500,28.7500,37.0500,31.1500,-12.4500,-7.3500,5.5500,-10.2500,0.237080600,5.29683300,1.067496000,1.9667860000,0.1134468701,0.0968307001,0.0418759016,0.0522975530,0.1134468701,0.0968307001,0.0418759016,0.0522975530,91.00,77.00,5.00,47.00,91.00,77.00,5.00,47.0,31.5600,11.2135250500,54.57202074,1.0
1,-0.6636760860,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,3.7140658366,6.3995308949,1.5598074021,3.3632779979,NaN,24.3400000000,36.1600000000,29.5200000000,NaN,24.34000,36.1600000000,29.5200000000,75.00,34.00,0.0,26.00,3.4668806371,2.6546632530,0.0321140671,0.4880867481,3.4668806371,2.6546632530,0.0321140671,0.4880867481,7.7811633032,31.1394527955,48.0177480655,21.9156825756,13.7900,8.7100,2.6900,6.3700,36.5000000000,36.5000000000,3.4668806371,2.6546632530,0.0321140671,0.4880867481,3.4668806371,2.6546632530,0.0321140671,0.4880867481,0.964835200,8.7679350,23.1592400,11.9620900,14.1500,28.7500,37.0500,31.1500,-12.4500,-7.3500,5.5500,-10.2500,0.237080600,5.29683300,1.067496000,1.9667860000,0.1134468701,0.0968307001,0.0418759016,0.0522975530,0.1134468701,0.0968307001,0.0418759016,0.0522975530,91.00,77.00,5.00,47.00,91.00,77.00,5.00,47.0,31.5600,11.2135250500,54.57202074,1.0
2,0.3478010620,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,2.1815202084,5.9723378265,5.0428776741,4.6374034668,13.920,26.5300000000,36.0800000000,NaN,13.920,26.53000,36.0800000000,NaN,79.00,26.00,0.0,13.00,0.3461917264,0.8982752558,0.0336629893,2.5013360811,0.3461917264,0.8982752558,0.0336629893,2.5013360811,8.1229049607,32.3882557036,48.1772426406,21.7575735702,2.2500,9.8100,9.3900,11.7500,13.2500000000,13.2500000000,0.3461917264,0.8982752558,0.0336629893,2.5013360811,0.3461917264,0.8982752558,0.0336629893,2.5013360811,3.334444000,10.5483700,23.2706500,11.5813200,17.0500,28.1500,37.5500,29.7500,-9.3500,-5.5500,1.2500,-7.25

In [45]:
loc

,Park,long,lat,veg,year,TimePeriod,RCP,treecanopy,Ann_Herb,Bare,Herb,Litter,Shrub,El,Sa,Cl,RF,Slope,E,S,T_P_Corr,DrySoilDays_Winter_top50,DrySoilDays_Spring_top50,DrySoilDays_Summer_top50,DrySoilDays_Fall_top50,DrySoilDays_Winter_whole,DrySoilDays_Spring_whole,DrySoilDays_Summer_whole,DrySoilDays_Fall_whole,Evap_Winter,Evap_Spring,Evap_Summer,Evap_Fall,ExtremeShortTermDryStress_Winter_top50,ExtremeShortTermDryStress_Spring_top50,ExtremeShortTermDryStress_Summer_top50,ExtremeShortTermDryStress_Fall_top50,ExtremeShortTermDryStress_Winter_whole,ExtremeShortTermDryStress_Spring_whole,ExtremeShortTermDryStress_Summer_whole,ExtremeShortTermDryStress_Fall_whole,FrostDays_Winter,FrostDays_Spring,FrostDays_Summer,FrostDays_Fall,NonDrySWA_Winter_top50,NonDrySWA_Spring_top50,NonDrySWA_Summer_top50,NonDrySWA_Fall_top50,NonDrySWA_Winter_whole,NonDrySWA_Spring_whole,NonDrySWA_Summer_whole,NonDrySWA_Fall_whole,PET_Winter,PET_Spring,PET_Summer,PET_Fall,PPT_Winter,PPT_Spring,PPT_Summer,PPT_Fall,SemiDryDuration_Annual_top50,SemiDryDuration_Annual_whole,SWA_Winter_top50,SWA_Spring_top50,SWA_Summer_top50,SWA_Fall_top50,SWA_Winter_whole,SWA_Spring_whole,SWA_Summer_whole,SWA_Fall_whole,T_Winter,T_Spring,T_Summer,T_Fall,Tmax_Winter,Tmax_Spring,Tmax_Summer,Tmax_Fall,Tmin_Winter,Tmin_Spring,Tmin_Summer,Tmin_Fall,Transp_Winter,Transp_Spring,Transp_Summer,Transp_Fall,VWC_Winter_top50,VWC_Spring_top50,VWC_Summer_top50,VWC_Fall_top50,VWC_Winter_whole,VWC_Spring_whole,VWC_Summer_whole,VWC_Fall_whole,WetSoilDays_Winter_top50,WetSoilDays_Spring_top50,WetSoilDays_Summer_top50,WetSoilDays_Fall_top50,WetSoilDays_Winter_whole,WetSoilDays_Spring_whole,WetSoilDays_Summer_whole,WetSoilDays_Fall_whole,PPT_Annual,T_Annual,RL,Location_ID
0,NABR,-110.0472,37.60413,Shrubland,1980,Hist,4.5,0,0,84,5,11,7,1764.955,77.03307,6.082058,2.285707,1949.283,-8753.784,4834.13,-0.6636760860,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,3.7140658366,6.3995308949,1.5598074021,3.3632779979,NaN,24.3400000000,36.1600000000,29.5200000000,NaN,24.34000,36.1600000000,29.5200000000,75.00,34.00,0.0,26.00,3.4668806371,2.6546632530,0.0321140671,0.4880867481,3.4668806371,2.6546632530,0.0321140671,0.4880867481,7.7811633032,31.1394527955,48.0177480655,21.9156825756,13.7900,8.7100,2.6900,6.3700,36.5000000000,36.5000000000,3.4668806371,2.6546632530,0.0321140671,0.4880867481,3.4668806371,2.6546632530,0.0321140671,0.4880867481,0.964835200,8.7679350,23.1592400,11.9620900,14.1500,28.7500,37.0500,31.1500,-12.4500,-7.3500,5.5500,-10.2500,0.237080600,5.29683300,1.067496000,1.9667860000,0.1134468701,0.0968307001,0.0418759016,0.0522975530,0.1134468701,0.0968307001,0.0418759016,0.0522975530,91.00,77.00,5.00,47.00,91.00,77.00,5.00,47.0,31.5600,11.2135250500,54.57202074,1.0
1,NABR,-110.0472,37.60413,Shrubland,1980,Hist,8.5,0,0,84,5,11,7,1764.955,77.03307,6.082058,2.285707,1949.283,-8753.784,4834.13,-0.6636760860,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,3.7140658366,6.3995308949,1.5598074021,3.3632779979,NaN,24.3400000000,36.1600000000,29.5200000000,NaN,24.34000,36.1600000000,29.5200000000,75.00,34.00,0.0,26.00,3.4668806371,2.6546632530,0.0321140671,0.4880867481,3.4668806371,2.6546632530,0.0321140671,0.4880867481,7.7811633032,31.1394527955,48.0177480655,21.9156825756,13.7900,8.7100,2.6900,6.3700,36.5000000000,36.5000000000,3.4668806371,2.6546632530,0.0321140671,0.4880867481,3.4668806371,2.6546632530,0.0321140671,0.4880867481,0.964835200,8.7679350,23.1592400,11.9620900,14.1500,28.7500,37.0500,31.1500,-12.4500,-7.3500,5.5500,-10.2500,0.237080600,5.29683300,1.067496000,1.9667860000,0.1134468701,0.0968307001,0.0418759016,0.0522975530,0.1134468701,0.0968307001,0.0418759016,0.0522975530,91.00,77.00,5.00,47.00,91.00,77.00,5.00,47.0,31.5600,11.2135250500,54.57202074,1.0
2,NABR,-110.0472,37.60413,Shrubland,1981,Hist,4.5,0,0,84,5,11,7,1764.955,77.03307,6.082058,2.285707,1949.283,-8753.784,4834.13,0.3478010620,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,2.1815202084,5.9723378265,5.0428776741,4.6374034668,13.920,26.5300000000,36.0800000000,NaN,13.

In [72]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns

temp = loc[loc['RCP']==4.5]

km = temp.dropna(axis=1, how='any')

# Fit KMeans model
n_clusters = 4
kmeans = KMeans(n_clusters=n_clusters, random_state=0)
kmeans.fit(km.iloc[:,20:])

# Add cluster labels to the data
km['Cluster'] = kmeans.labels_
km['Cluster'] = km['Cluster'].astype('category')


In [73]:
# Create the scatter mapbox

map = df_con[df_con['year']==2099].groupby(['long','lat','veg'])['T_Annual'].mean().reset_index()

fig = px.scatter_mapbox(map, lat="lat", lon="long", color="veg",
                        color_continuous_scale=px.colors.cyclical.IceFire, size_max=8, zoom=11,
                        mapbox_style="open-street-map")

# Update the layout with the new legend title and position
fig.update_layout(
    title={
        'text': "<b>Vegetation Location</b>",
        'y': 0.97,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    coloraxis_colorbar={
        'title': 'Vegetation Level'  # Change this to your desired legend title
    },
    legend={
        'x': 1,  # Position the legend to the right
        'y': 0.8,  # Center the legend vertically
        'xanchor': 'left',  # Anchor the legend's x position to the left side
        'yanchor': 'middle'  # Anchor the legend's y position to the middle
    },
    margin={"r": 0, "t": 40, "l": 0, "b": 0}
)
fig.update_traces(marker=dict(size=10))  # Set the desired fixed marker size

# Show the figure
fig.show()

In [77]:
# map = km.groupby(['long','lat','Cluster'])['T_Annual'].mean().reset_index()
map = km

fig = px.scatter_mapbox(map, lat="lat", lon="long", color="Cluster",
                        # color_continuous_scale=px.colors.cyclical.IceFire, 
                        size_max=8, zoom=11,
                        mapbox_style="open-street-map")

# Update the layout with the new legend title and position
fig.update_layout(
    title={
        'text': "<b>Vegetation Location</b>",
        'y': 0.97,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    coloraxis_colorbar={
        'title': 'Vegetation Level'  # Change this to your desired legend title
    },
    legend={
        'x': 1,  # Position the legend to the right
        'y': 0.8,  # Center the legend vertically
        'xanchor': 'left',  # Anchor the legend's x position to the left side
        'yanchor': 'middle'  # Anchor the legend's y position to the middle
    },
    margin={"r": 0, "t": 40, "l": 0, "b": 0}
)
fig.update_traces(marker=dict(size=10))  # Set the desired fixed marker size

# Show the figure
fig.show()

In [83]:
# Examine cluster centroids
centroids = pd.DataFrame(kmeans.cluster_centers_, columns=km.iloc[:,20:].columns[:-1])
print("Cluster Centroids:")
centroids

Cluster Centroids:


,DrySoilDays_Fall_top50,DrySoilDays_Winter_whole,DrySoilDays_Spring_whole,DrySoilDays_Summer_whole,DrySoilDays_Fall_whole,Evap_Winter,Evap_Spring,Evap_Summer,Evap_Fall,ExtremeShortTermDryStress_Winter_top50,ExtremeShortTermDryStress_Spring_top50,ExtremeShortTermDryStress_Summer_top50,ExtremeShortTermDryStress_Fall_top50,ExtremeShortTermDryStress_Winter_whole,ExtremeShortTermDryStress_Spring_whole,ExtremeShortTermDryStress_Summer_whole,ExtremeShortTermDryStress_Fall_whole,FrostDays_Winter,FrostDays_Spring,FrostDays_Summer,FrostDays_Fall,NonDrySWA_Winter_top50,NonDrySWA_Spring_top50,NonDrySWA_Summer_top50,NonDrySWA_Fall_top50,NonDrySWA_Winter_whole,NonDrySWA_Spring_whole,NonDrySWA_Summer_whole,NonDrySWA_Fall_whole,PET_Winter,PET_Spring,PET_Summer,PET_Fall,PPT_Winter,PPT_Spring,PPT_Summer,PPT_Fall,SemiDryDuration_Annual_top50,SemiDryDuration_Annual_whole,SWA_Winter_top50,SWA_Spring_top50,SWA_Summer_top50,SWA_Fall_top50,SWA_Winter_whole,SWA_Spring_whole,SWA_Summer_whole,SWA_Fall_whole,T_Winter,T_Spring,T_Summer,T_Fall,Tmax_Winter,Tmax_Spring,Tmax_Summer,Tmax_Fall,Tmin_Winter,Tmin_Spring,Tmin_Summer,Tmin_Fall,Transp_Winter,Transp_Spring,Transp_Summer,Transp_Fall,VWC_Winter_top50,VWC_Spring_top50,VWC_Summer_top50,VWC_Fall_top50,VWC_Winter_whole,VWC_Spring_whole,VWC_Summer_whole,VWC_Fall_whole,WetSoilDays_Winter_top50,WetSoilDays_Spring_top50,WetSoilDays_Summer_top50,WetSoilDays_Fall_top50,WetSoilDays_Winter_whole,WetSoilDays_Spring_whole,WetSoilDays_Summer_whole,WetSoilDays_Fall_whole,PPT_Annual,T_Annual,RL,Location_ID
0,3.6951248717,9.8563120082e-02,9.8323640096e-02,1.0432603490e+00,9.3600752651e-01,2.1767183234,4.5077724287,3.6493963493,3.3819213049,12.0113876799,29.8457621963,37.3288298273,31.1933853519,12.0068416520,29.8585237917,37.3282353591,31.1905557316,65.4388812863,10.8464249059,1.0263427985e-04,15.3991960315,3.5393427968,2.5251518070,0.2488037701,1.7353179176,4.9037210224,3.8081917540,0.2853220727,2.2956895171,5.1678779758,31.8145576252,49.1278039131,18.4197827409,7.2603588779,6.5244450907,8.1307632569,9.6625246322,28.5612008704,28.5452307219,3.5379709756,2.5222653945,0.2227233070,1.7144839523,4.9028160997,3.8070589439,0.2820172091,2.2890032014,2.2305107184,12.4121702864,24.9088480773,13.3942890908,16.6544284981,32.0689555251,38.5293720493,33.1236455696,-12.5106344509,-4.0828398905,7.4975443038,-7.2448658912,0.5335901367,6.2758200479,3.8674322434,2.1689957062,0.1467047276,0.1255482321,0.0769834825,0.1086483808,0.1441741926,0.1273277117,0.0771792201,0.1066263791,85.9377352036,71.4326890181,16.5431406090,60.9490420801,85.9488539172,71.9676873076,16.7371707150,61.0507526514,31.5780918577,13.2364545432,76.9585235135,70.4594594595
1,5.9165781952,4.6263781135e-02,5.3572886893e-02,1.8770722744e+00,6.6778276848e-01,1.8837023562,4.5346011203,3.6687023112,3.1633520351,11.4801972726,29.5498021494,36.7170725969,30.6901628545,11.4726609322,29.5974612333,36.6867014740,30.7003294819,67.3045120457,12.6158227848,6.5332788893e-04,16.3816251531,3.5599128181,2.9887812859,0.3623834773,1.8256589182,6.7205190318,6.6692382742,0.7128847735,3.1570007173,4.2768219511,30.6158921737,48.2461473695,16.7898701039,7.6559132299,6.8754932626,8.6458503471,10.0506284198,26.4910903357,26.0267035098,3.5577917571,2.9850381820,0.3183710616,1.7928399249,6.7202162485,6.6688017282,0.7071879021,3.1498618767,1.8984084490,11.9559599600,24.4491335020,12.9932174550,16.2278478971,31.4690426705,37.9415242956,32.5800471621,-13.0118648428,-4.5187415272,7.1448742344,-7.7035890159,0.4247877115,7.3977300907,5.8126561124,2.0252997128,0.1510036927,0.1391171540,0.0834214702,0.1144395444,0.1404428998,0.1392732808,0.0823334202,0.1062867328,87.0323805635,78.7366884443,22.1698448346,64.0398121682,87.0798693344,80.4059003675,24.6947529604,64.3222743977,33.2278852593,12.8241798415,133.2520267742,81.9354838710
2,0.4328369322,5.5511151231e-17,6.6613381478e-16,1.7763568394e-15,8.8817841970e-16,2.6354840871,4.1736001243,3.5017645378,3.6300183166,12.4557168690,30.1393406121,37.7635734922,31

In [96]:
rank_df = centroids.rank().reset_index()

In [97]:
long_df = pd.melt(rank_df, id_vars=['index'], value_vars=list(rank_df.columns[1:]), 
                  var_name='Feature', value_name='Rank')

In [102]:
long_df[(long_df['index'] == 2) & (long_df['Rank'] == 1)]

,index,Feature,Rank
2,2,DrySoilDays_Fall_top50,1.0
6,2,DrySoilDays_Winter_whole,1.0
10,2,DrySoilDays_Spring_whole,1.0
14,2,DrySoilDays_Summer_whole,1.0
18,2,DrySoilDays_Fall_whole,1.0
26,2,Evap_Spring,1.0
30,2,Evap_Summer,1.0
70,2,FrostDays_Winter,1.0
74,2,FrostDays_Spring,1.0
78,2,FrostDays_Summer,1.0


In [114]:
selected_columns = [col for col in df.columns if not col.startswith(('T_', 'Tmin', 'Tmax'))]
filtered_df = df_orig[selected_columns]


In [115]:
filtered_df['T_Annual'] = df_orig['T_Annual']

In [116]:
#| code-summary: PCA(RCP = 4.5)
data = filtered_df[(filtered_df['RCP']==4.5) & (filtered_df['year'].isin(range(2095,2100)))].dropna(axis=1, how='any')
X = data.iloc[:,list(range(1, 3))+ [4,6] + list(range(8, len(data.columns)-3))]

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply PCA
pca = PCA(n_components=10)  # Reduce to 2 components for visualization
X_pca = pca.fit_transform(X_scaled)

# Add the PCA and cluster results to the DataFrame
data['PCA1'] = X_pca[:, 0]
data['PCA2'] = X_pca[:, 1]
data['PCA3'] = X_pca[:, 2]

# Get the component loadings
loadings = pca.components_.T
columns = X.columns

# Create a DataFrame for loadings
loadings_df = pd.DataFrame(loadings, columns=['PCA1', 'PCA2', 'PCA3', 'PCA4', 'PCA5', 'PCA6', 'PCA7', 'PCA8', 'PCA9', 'PCA10'], index=columns)

# Get the explained variance ratio
explained_variance_ratio = pca.explained_variance_ratio_

# Cumulative explained variance
cumulative_explained_variance = np.cumsum(explained_variance_ratio)

In [117]:
#| code-summary: Varience Ratio

# Create a list of x-axis labels from PCA1 to PCA10
x_labels = [f'PCA{i+1}' for i in range(len(explained_variance_ratio))]

# Create a line chart for explained variance ratio
fig = go.Figure(data=go.Scatter(
    x=x_labels,
    y=explained_variance_ratio,
    mode='lines+markers',
    text=explained_variance_ratio,
    textposition='top center'
))

# Update layout for better readability
fig.update_layout(
    title='<b>Explained Variance Ratio by Principal Components</b>',
    xaxis_title='Principal Components',
    yaxis_title='Explained Variance Ratio',
    yaxis=dict(tickformat=".2%", range=[0, 1.1 * explained_variance_ratio.max()]),  # Adjust the range as needed
    template='plotly_white',
    margin=dict(l=50, r=50, b=50, t=50)  # Adjust the padding as needed
)

fig.show()

In [118]:
#| code-summary: Feature Importance Plots

# Sort the features by the absolute value of the loading for PCA1
sorted_loadings = loadings_df['PCA1'].abs().sort_values(ascending=False)

# Get the top 10 most influential features
top_features = sorted_loadings.head(30).index

# Get the actual loadings for these top 10 features
top_loadings = round(loadings_df.loc[top_features, 'PCA1'],4)

# Create a color list based on the sign of the loadings
colors = ['blue' if val > 0 else 'red' for val in top_loadings]

# Create a bar chart
fig = go.Figure(data=[go.Bar(
    x=top_loadings.index,
    y=top_loadings.abs(),
    text=top_loadings.values,  # Show the actual values as text
    textposition='inside',
    marker_color=colors,
    showlegend=False
)])

# Add legend manually
fig.add_trace(go.Bar(
    x=[None], y=[None],
    marker=dict(color='blue'),
    showlegend=True,
    name='Positive'
))
fig.add_trace(go.Bar(
    x=[None], y=[None],
    marker=dict(color='red'),
    showlegend=True,
    name='Negative'
))

# Update layout for better readability
fig.update_layout(
    title='<b>Top 20 Most Influential Features on PCA1</b>',
    xaxis_title='Features',
    yaxis_title='Absolute PCA1 Loadings',
    yaxis=dict(tickformat=".2f"),
    template='plotly_white',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    margin=dict(l=20, r=20, b=20, t=60)
)

fig.show()

# Sort the features by the absolute value of the loading for PCA1
sorted_loadings = loadings_df['PCA2'].abs().sort_values(ascending=False)

# Get the top 10 most influential features
top_features = sorted_loadings.head(30).index

# Get the actual loadings for these top 10 features
top_loadings = round(loadings_df.loc[top_features, 'PCA2'],4)

# Create a color list based on the sign of the loadings
colors = ['blue' if val > 0 else 'red' for val in top_loadings]

# Create a bar chart
fig = go.Figure(data=[go.Bar(
    x=top_loadings.index,
    y=top_loadings.abs(),
    text=top_loadings.values,  # Show the actual values as text
    textposition='inside',
    marker_color=colors,
    showlegend=False
)])

# Add legend manually
fig.add_trace(go.Bar(
    x=[None], y=[None],
    marker=dict(color='blue'),
    showlegend=True,
    name='Positive'
))
fig.add_trace(go.Bar(
    x=[None], y=[None],
    marker=dict(color='red'),
    showlegend=True,
    name='Negative'
))

# Update layout for better readability
fig.update_layout(
    title='<b>Top 20 Most Influential Features on PCA2</b>',
    xaxis_title='Features',
    yaxis_title='Absolute PCA1 Loadings',
    yaxis=dict(tickformat=".2f"),
    template='plotly_white',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    margin=dict(l=20, r=20, b=20, t=60)
)

fig.show()

# Sort the features by the absolute value of the loading for PCA1
sorted_loadings = loadings_df['PCA3'].abs().sort_values(ascending=False)

# Get the top 10 most influential features
top_features = sorted_loadings.head(30).index

# Get the actual loadings for these top 10 features
top_loadings = round(loadings_df.loc[top_features, 'PCA3'],4)

# Create a color list based on the sign of the loadings
colors = ['blue' if val > 0 else 'red' for val in top_loadings]

# Create a bar chart
fig = go.Figure(data=[go.Bar(
    x=top_loadings.index,
    y=top_loadings.abs(),
    text=top_loadings.values,  # Show the actual values as text
    textposition='inside',
    marker_color=colors,
    showlegend=False
)])

# Add legend manually
fig.add_trace(go.Bar(
    x=[None], y=[None],
    marker=dict(color='blue'),
    showlegend=True,
    name='Positive'
))
fig.add_trace(go.Bar(
    x=[None], y=[None],
    marker=dict(color='red'),
    showlegend=True,
    name='Negative'
))

# Update layout for better readability
fig.update_layout(
    title='<b>Top 20 Most Influential Features on PCA3</b>',
    xaxis_title='Features',
    yaxis_title='Absolute PCA1 Loadings',
    yaxis=dict(tickformat=".2f"),
    template='plotly_white',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    margin=dict(l=20, r=20, b=20, t=60)
)

fig.show()

In [119]:
#| code-summary: 2D PCA Plots

# Visualize the results with Plotly
fig = px.scatter(
    data,
    x='PCA1',
    y='PCA2',
    color='scenario',
    title='<b>PCA For All Scenarios (RCP = 4.5)</b>',
    labels={'PCA1': 'PCA1', 'PCA2': 'PCA2'},
    opacity=0.5
)

fig.update_layout(
    margin=dict(l=10, r=10, b=10, t=40),  # Adjust the values as needed
    title_x=0.5,
    title_y=0.95,
)

fig.show()

fig = px.scatter(
    data[data['scenario'].isin(['sc37','sc40'])],
    x='PCA1',
    y='PCA2',
    color='scenario',
    title='<b>PCA for Scenario 37 vs 40 (RCP = 4.5)</b>',
    labels={'PCA1': 'PCA1', 'PCA2': 'PCA2'},
    opacity=0.5
)

fig.update_layout(
    margin=dict(l=10, r=10, b=10, t=40),  # Adjust the values as needed
    title_x=0.5,
    title_y=0.95,
)

fig.show()

In [121]:
#| code-summary: 3D PCA Plots

# Visualize the results with Plotly in 3D
fig = px.scatter_3d(
    data,
    x='PCA1',
    y='PCA2',
    z='PCA3',
    color='scenario',
    title='<b>3D PCA For All Scenarios</b>',
    labels={'PCA1': 'PCA1', 'PCA2': 'PCA2', 'PCA3': 'PCA3'},
    opacity=0.5,
    size_max=0.1
)

fig.update_traces(marker=dict(size=3))  # Adjust the size value as needed


# Update layout to adjust padding and margins
fig.update_layout(
    margin=dict(l=5, r=5, b=5, t=20),  # Adjust the values as needed
    title_x=0.5,
    title_y=0.95,
    scene=dict(
        xaxis=dict(title='PCA1'),
        yaxis=dict(title='PCA2'),
        zaxis=dict(title='PCA3'),
                camera=dict(
            eye=dict(x=0, y=2, z=0)
                )
    )
)
fig.show()

fig = px.scatter_3d(
    data[data['scenario'].isin(['sc37','sc40'])],
    x='PCA1',
    y='PCA2',
    z='PCA3',
    color='scenario',
    title='<b>3D PCA for Scenario 37 vs 40</b>',
    labels={'PCA1': 'PCA1', 'PCA2': 'PCA2', 'PCA3': 'PCA3'},
    opacity=0.5,
    size_max=0.1
)

fig.update_traces(marker=dict(size=3))  # Adjust the size value as needed


# Update layout to adjust padding and margins
fig.update_layout(
    margin=dict(l=5, r=5, b=5, t=20),  # Adjust the values as needed
    title_x=0.5,
    title_y=0.95,
    scene=dict(
        xaxis=dict(title='PCA1'),
        yaxis=dict(title='PCA2'),
        zaxis=dict(title='PCA3'),
                camera=dict(
            eye=dict(x=0, y=2, z=0)
                )
    )
)
fig.show()